# Xtremax - Data Engineer Test

**Author:** [Muhammad Ilham Siddiqqulhakim](https://www.linkedin.com/in/milhams/)

**Dataset:** [120 Years of Olympic History](https://mavenanalytics.io/data-playground)

**Description:** An ETL Code using [120 Years of Olympic History](https://mavenanalytics.io/data-playground)
- Script must contain data preparation/cleaning step
- Script must contain data summarization step
- Make sure script are running properly
- Clearly written (utilize comment)

## Data Preparation/Cleaning

### Dependencies

In [1]:
# import dependencies
import pyodbc
import pandas as pd

### Load data

In [2]:
# Database connection
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost,1433;'
    'DATABASE=olympic_athletes;'
    'Trusted_Connection=yes;'
)

# Query to fetch the data
athlete_query = "SELECT * FROM athlete_events"
country_query = "SELECT * FROM country_definitions"

# Load data into pandas DataFrame
athlete_events_df = pd.read_sql(athlete_query, conn)
country_def_df = pd.read_sql(country_query, conn)

print(athlete_events_df.head())
print(country_def_df.head())

C:\Users\siddi\AppData\Local\Temp\ipykernel_2724\543578673.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  athlete_events_df = pd.read_sql(athlete_query, conn)


   ID                      Name Sex Age Height Weight            Team  NOC  \
0   1                 A Dijiang   M  24    180     80           China  CHN   
1   2                  A Lamusi   M  23    170     60           China  CHN   
2   3       Gunnar Nielsen Aaby   M  24     NA     NA         Denmark  DEN   
3   4      Edgar Lindenau Aabye   M  34     NA     NA  Denmark/Sweden  DEN   
4   5  Christine Jacoba Aaftink   F  21    185     82     Netherlands  NED   

         Games  Year  Season       City          Sport  \
0  1992 Summer  1992  Summer  Barcelona     Basketball   
1  2012 Summer  2012  Summer     London           Judo   
2  1920 Summer  1920  Summer  Antwerpen       Football   
3  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal  
0       Basketball Men's Basketball    NA  
1      Judo Men's Extra-Lightweight    NA  
2           Football Men's Football    NA  
3     

C:\Users\siddi\AppData\Local\Temp\ipykernel_2724\543578673.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  country_def_df = pd.read_sql(country_query, conn)


### Cleansing/Preparation

In [3]:
# 1. Handle Missing Values
# Replace 'NA' strings with actual NaN values for proper handling
athlete_events_df.replace('NA', pd.NA, inplace=True)

In [4]:
# 2. Data Type Conversion
# Convert numerical columns to the appropriate data types
athlete_events_df['Age'] = pd.to_numeric(athlete_events_df['Age'], errors='coerce')
athlete_events_df['Height'] = pd.to_numeric(athlete_events_df['Height'], errors='coerce')
athlete_events_df['Weight'] = pd.to_numeric(athlete_events_df['Weight'], errors='coerce')

In [5]:
# 3. Remove Duplicates
# Drop duplicate records
athlete_events_df.drop_duplicates(inplace=True)

In [6]:
# 4. Join with country definition data
# Merge the two datasets to get the region information
merged_df = pd.merge(athlete_events_df, country_def_df, on='NOC', how='left')

In [7]:
merged_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,<NA>,China,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,<NA>,China,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,<NA>,Denmark,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,None
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,<NA>,Netherlands,None


Since the data is already "good", so I just need to do simple cleansing/transformation

## Data Summarization

In [8]:
# Metric 1: Medal Summary by Country and Type
medal_summary = merged_df.groupby(['region', 'Medal']).size().unstack(fill_value=0)
print("Medal Summary by Country and Type:")
print(medal_summary)

Medal Summary by Country and Type:
Medal               Bronze  Gold  Silver
region                                  
Afghanistan              2     0       0
Algeria                  8     5       4
Argentina               91    91      92
Armenia                  9     2       5
Australia              522   368     459
...                    ...   ...     ...
Venezuela               10     2       3
Vietnam                  0     1       3
Virgin Islands, US       0     0       1
Zambia                   1     0       1
Zimbabwe                 1    17       4

[136 rows x 3 columns]


In [9]:
# Metric 2: Average Age, Height, and Weight by Country
athlete_summary = merged_df.groupby('region').agg({
    'Age': 'mean',
    'Height': 'mean',
    'Weight': 'mean'
}).reset_index()
print("Athlete Summary by Country (Average Age, Height, Weight):")
print(athlete_summary)

Athlete Summary by Country (Average Age, Height, Weight):
                      region        Age      Height     Weight
0                Afghanistan  23.538462  170.592593  65.901639
1                    Albania  25.342857  173.000000  71.491803
2                    Algeria  24.370642  174.702869  68.693252
3             American Samoa  27.216216  175.666667  87.958333
4                    Andorra  23.065089  173.703704  70.644444
..                       ...        ...         ...        ...
201  Virgin Islands, British  26.200000  178.631579  73.800000
202       Virgin Islands, US  27.336770  177.933333  73.412500
203                    Yemen  21.604167  168.189189  61.526316
204                   Zambia  23.461039  172.828125  66.035971
205                 Zimbabwe  25.211838  173.203333  68.049834

[206 rows x 4 columns]


In [10]:
# Metric 3: Gender Distribution by Country
gender_summary = merged_df.groupby(['region', 'Sex']).size().unstack(fill_value=0)
print("Gender Distribution by Country:")
print(gender_summary)

Gender Distribution by Country:
Sex                        F    M
region                           
Afghanistan                5  121
Albania                   27   43
Algeria                   94  457
American Samoa             7   30
Andorra                   42  127
...                      ...  ...
Virgin Islands, British    5   40
Virgin Islands, US        57  237
Yemen                      4   44
Zambia                    19  164
Zimbabwe                 114  209

[206 rows x 2 columns]


In [11]:
# Metric 4: Total Number of Athletes Participating in Each Sport by Country
sport_summary = merged_df.groupby(['region', 'Sport']).size().unstack(fill_value=0)
print("Total Number of Athletes Participating in Each Sport by Country:")
print(sport_summary)

Total Number of Athletes Participating in Each Sport by Country:
Sport                    Aeronautics  Alpine Skiing  Alpinism  Archery  \
region                                                                   
Afghanistan                        0              0         0        0   
Albania                            0              7         0        0   
Algeria                            0             10         0        0   
American Samoa                     0              0         0        1   
Andorra                            0            105         0        0   
...                              ...            ...       ...      ...   
Virgin Islands, British            0              0         0        0   
Virgin Islands, US                 0              9         0        0   
Yemen                              0              0         0        0   
Zambia                             0              0         0        0   
Zimbabwe                           0           

## Finishing

In [12]:
# Add Age Category For Dashboard
bins = [0, 18, 25, 35, 45, 100]
labels = ['0-18', '19-25', '26-35', '36-45', '46+']
merged_df['Age Category'] = pd.cut(merged_df['Age'], bins=bins, labels=labels, right=False)
print(merged_df[['Age', 'Age Category']])

         Age Age Category
0       24.0        19-25
1       23.0        19-25
2       24.0        19-25
3       34.0        26-35
4       21.0        19-25
...      ...          ...
269726  29.0        26-35
269727  27.0        26-35
269728  27.0        26-35
269729  30.0        26-35
269730  34.0        26-35

[269731 rows x 2 columns]


In [13]:
# Export to CSV for Power BI
merged_df.to_csv("olympic_athletes_merged.csv", index=False)

In [14]:
# Close the connection
conn.close()